In [1]:
import cv2 as cv 
import numpy as np
import sys
from PyQt5.QtWidgets import *

# [1] 메인 윈도우 클래스 정의
class Orim(QMainWindow):
    def __init__(self):
        super().__init__()  # QMainWindow 초기화

        self.setWindowTitle('오림')  # 윈도우 제목
        self.setGeometry(200, 200, 700, 200)  # 위치(x,y)와 크기(width, height)

        # [2] 버튼 7개 생성
        fileButton = QPushButton('파일', self)
        paintButton = QPushButton('페인팅', self)
        cutButton = QPushButton('오림', self)
        incButton = QPushButton('+', self)
        decButton = QPushButton('-', self)
        saveButton = QPushButton('저장', self)
        quitButton = QPushButton('나가기', self)

        # [3] 버튼 위치 배치
        fileButton.setGeometry(10, 10, 100, 30)
        paintButton.setGeometry(110, 10, 100, 30)
        cutButton.setGeometry(210, 10, 100, 30)
        incButton.setGeometry(310, 10, 50, 30)
        decButton.setGeometry(360, 10, 50, 30)
        saveButton.setGeometry(410, 10, 100, 30)
        quitButton.setGeometry(510, 10, 100, 30)

        # [4] 버튼 클릭 시 실행될 함수 연결 (Signal-Slot)
        fileButton.clicked.connect(self.fileOpenFunction)
        paintButton.clicked.connect(self.paintFunction)
        cutButton.clicked.connect(self.cutFunction)
        incButton.clicked.connect(self.incFunction)
        decButton.clicked.connect(self.decFunction)
        saveButton.clicked.connect(self.saveFunction)
        quitButton.clicked.connect(self.quitFunction)

        # [5] 페인팅 기본값
        self.BrushSiz = 5                   # 붓 크기 초기값
        self.LColor, self.RColor = (255, 0, 0), (0, 0, 255)  # 파랑=물체, 빨강=배경

    # [6] 파일 열기 함수
    def fileOpenFunction(self):
        fname = QFileDialog.getOpenFileName(self, 'Open file', './')  # 파일 선택
        self.img = cv.imread(fname[0])  # 이미지 읽기
        if self.img is None:
            sys.exit('파일을 찾을 수 없습니다.')

        self.img_show = np.copy(self.img)  # 표시용 이미지 (붓칠 표시)
        cv.imshow('Painting', self.img_show)

        # GrabCut용 마스크 초기화
        self.mask = np.zeros((self.img.shape[0], self.img.shape[1]), np.uint8)
        self.mask[:, :] = cv.GC_PR_BGD  # 모든 화소를 잠재적 배경으로 초기화

    # [7] 페인팅 함수: 마우스 이벤트 콜백 연결
    def paintFunction(self):
        cv.setMouseCallback('Painting', self.painting)

    # [8] 실제 마우스 드로잉 함수
    def painting(self, event, x, y, flags, param):
        if event == cv.EVENT_LBUTTONDOWN:
            # 왼쪽 버튼 클릭 → 파란색 → 물체 영역
            cv.circle(self.img_show, (x, y), self.BrushSiz, self.LColor, -1)
            cv.circle(self.mask, (x, y), self.BrushSiz, cv.GC_FGD, -1)  # 확실한 전경

        elif event == cv.EVENT_RBUTTONDOWN:
            # 오른쪽 버튼 클릭 → 빨간색 → 배경 영역
            cv.circle(self.img_show, (x, y), self.BrushSiz, self.RColor, -1)
            cv.circle(self.mask, (x, y), self.BrushSiz, cv.GC_BGD, -1)  # 확실한 배경

        elif event == cv.EVENT_MOUSEMOVE and flags == cv.EVENT_FLAG_LBUTTON:
            # 왼쪽 버튼 누른 채 이동 → 파란색 그리기
            cv.circle(self.img_show, (x, y), self.BrushSiz, self.LColor, -1)
            cv.circle(self.mask, (x, y), self.BrushSiz, cv.GC_FGD, -1)

        elif event == cv.EVENT_MOUSEMOVE and flags == cv.EVENT_FLAG_RBUTTON:
            # 오른쪽 버튼 누른 채 이동 → 빨간색 그리기
            cv.circle(self.img_show, (x, y), self.BrushSiz, self.RColor, -1)
            cv.circle(self.mask, (x, y), self.BrushSiz, cv.GC_BGD, -1)

        cv.imshow('Painting', self.img_show)

    # [9] GrabCut으로 전경/배경 분리 (오림)
    def cutFunction(self):
        background = np.zeros((1, 65), np.float64)  # 배경 GMM
        foreground = np.zeros((1, 65), np.float64)  # 전경 GMM

        cv.grabCut(
            self.img,         # 원본 이미지
            self.mask,        # 사용자 붓칠 마스크
            None,             # ROI 없음
            background,
            foreground,
            5,                # 반복 횟수
            cv.GC_INIT_WITH_MASK  # 마스크 기반으로 초기화
        )

        # 확실한/잠재적 배경은 0, 나머지는 1로
        mask2 = np.where((self.mask == 2) | (self.mask == 0), 0, 1).astype('uint8')
        self.grabImg = self.img * mask2[:, :, np.newaxis]  # 전경만 남김

        cv.imshow('Scissoring', self.grabImg)  # 결과 출력

    # [10] 붓 크기 증가
    def incFunction(self):
        self.BrushSiz = min(20, self.BrushSiz + 1)

    # [11] 붓 크기 감소
    def decFunction(self):
        self.BrushSiz = max(1, self.BrushSiz - 1)

    # [12] 결과 이미지 저장
    def saveFunction(self):
        fname = QFileDialog.getSaveFileName(self, '파일 저장', './')
        cv.imwrite(fname[0], self.grabImg)

    # [13] 종료
    def quitFunction(self):
        cv.destroyAllWindows()
        self.close()

# [14] PyQt5 앱 실행
app = QApplication(sys.argv)
win = Orim()
win.show()
app.exec_()


0